In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [6]:
## Define train and test paths
train_path = './FRE2013/train'
test_path = './FRE2013/test'

In [10]:
from datetime import datetime
from packaging import version

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,MaxPool2D,Conv2D,Dropout,MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [13]:
%load_ext tensorboard
import tensorboard
tensorboard.__version__

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


'2.9.1'

In [14]:
!rm -rf ./logs/

In [15]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [16]:
train_set=ImageDataGenerator().flow_from_directory(train_path,
                                             target_size=(48,48),
                                             batch_size=32,
                                             color_mode='grayscale',
                                             class_mode='categorical')
test_set=ImageDataGenerator().flow_from_directory(test_path,
                                             target_size=(48,48),
                                             batch_size=32,
                                             color_mode='grayscale',
                                             class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [29]:
filter_size = 2
num_convolutions = 128 
num_strides = 2

In [38]:
model=Sequential()

model.add(Conv2D(num_convolutions,(filter_size,filter_size),activation="relu",input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(2,2),strides=num_strides))

model.add(Conv2D(num_convolutions,(filter_size,filter_size),activation="relu",input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(2,2),strides=num_strides))

model.add(Conv2D(num_convolutions,(filter_size,filter_size),activation="relu",input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(2,2),strides=num_strides))

model.add(Conv2D(num_convolutions,(filter_size,filter_size),activation="relu",input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(2,2),strides=num_strides))

model.add(Flatten())
model.add(tf.keras.layers.Dense(124,activation='relu'))
model.add(tf.keras.layers.Dropout(0.15))
  #Dense 2
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
  
  #Output Layer
model.add(tf.keras.layers.Dense(7,activation = 'softmax'))
#model.add(Dense(256, activation='tanh'))
#model.add(Dropout(0.25))

#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.25))

    
# model.add(Dense(7, activation='softmax'))

In [39]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 47, 47, 128)       640       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 23, 23, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 22, 22, 128)       65664     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 11, 11, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 10, 10, 128)       65664     
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 5, 5, 128)       

In [40]:
history=model.fit(train_set,epochs=120,validation_data=test_set,batch_size=32,callbacks=[tensorboard_callback])

Epoch 1/120
898/898 [==============================] - 12s 13ms/step - loss: 1.8394 - accuracy: 0.2446 - val_loss: 1.7617 - val_accuracy: 0.2749
Epoch 2/120
898/898 [==============================] - 12s 13ms/step - loss: 1.6956 - accuracy: 0.3114 - val_loss: 1.5412 - val_accuracy: 0.3965
Epoch 3/120
898/898 [==============================] - 12s 13ms/step - loss: 1.5255 - accuracy: 0.3990 - val_loss: 1.4297 - val_accuracy: 0.4452
Epoch 4/120
898/898 [==============================] - 12s 13ms/step - loss: 1.4208 - accuracy: 0.4473 - val_loss: 1.3812 - val_accuracy: 0.4599
Epoch 5/120
898/898 [==============================] - 12s 13ms/step - loss: 1.3563 - accuracy: 0.4801 - val_loss: 1.3658 - val_accuracy: 0.4688
Epoch 6/120
898/898 [==============================] - 12s 13ms/step - loss: 1.3090 - accuracy: 0.4955 - val_loss: 1.3130 - val_accuracy: 0.4930
Epoch 7/120
898/898 [==============================] - 12s 13ms/step - loss: 1.2696 - accuracy: 0.5161 - val_loss: 1.2867 - val_ac

KeyboardInterrupt: 